In [1]:
import torch
import numpy as np
import phi.torch.flow as ptf

import tqdm

device = torch.device('cuda')

In [ ]:
BOUNDS = ptf.Box(x=1, y=1, z=1)
RESOLUTION = 100

In [8]:
index1 = 0
index2 = index1 + 5
den1_np = np.load(f'origin/origin_{index1:03d}.npz')['density']
den2_np = np.load(f'origin/origin_{index2:03d}.npz')['density']
den1_gpu = torch.tensor(den1_np, dtype=torch.float32, device=device)
den2_gpu = torch.tensor(den2_np, dtype=torch.float32, device=device)

DENSITY1 = ptf.CenteredGrid(
    values=ptf.wrap(den1_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
DENSITY2 = ptf.CenteredGrid(
    values=ptf.wrap(den2_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
VELOCITY = ptf.StaggeredGrid(
    values=0,
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
INFLOW = ptf.CenteredGrid(
    values=ptf.Sphere(center=ptf.tensor([0.5, 0, 0.5], ptf.channel(vector='x,y,z')), radius=0.05),
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)


# @ptf.jit_compile
def step(v, s, p, dt, inflow_rate, inflow, FORCE):
    s = ptf.advect.mac_cormack(s, v, dt) + inflow_rate * ptf.resample(inflow, to=s, soft=True)
    buoyancy = ptf.resample(s * (0, 0.1, 0), to=v) + FORCE
    v = ptf.advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = ptf.fluid.make_incompressible(v, (), ptf.Solve('auto', 1e-3, x0=p))
    return v, s, p

In [ ]:
for _ in tqdm.trange(100):

In [7]:
x = torch.linspace(0, 1, 5, dtype=torch.float32, device=device)
y = torch.linspace(0, 1, 5, dtype=torch.float32, device=device)
z = torch.linspace(0, 1, 5, dtype=torch.float32, device=device)
grid_x, grid_y, grid_z = torch.meshgrid(x, y, z, indexing='ij')
grid_points = torch.stack([grid_x, grid_y, grid_z], dim=-1)

TO_LEARN_DIRECTION = []
LOCATIONS = []
DIRECTIONS = []
GRAD_vars = []
for pos in grid_points.reshape(-1, 3):
    TO_LEARN_DIRECTION.append(torch.randn(3, device=device, requires_grad=True))
    LOCATIONS.append(ptf.vec(x=pos[0], y=pos[1], z=pos[2]))
    DIRECTIONS.append(ptf.wrap(TO_LEARN_DIRECTION[-1], ptf.channel(vector='x,y,z')))
    GRAD_vars.append(TO_LEARN_DIRECTION[-1])
OPTIMIZER = torch.optim.RAdam(GRAD_vars, lr=0.001)

FORCE_FIELD = ptf.CenteredGrid(
    values=(0, 0, 0),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)

# Gaussian falloff function
A = 0.2  # 高斯幅值
sigma = 0.2  # 标准差，控制衰减范围
for LOCATION, DIRECTION in tqdm.tqdm(zip(LOCATIONS, DIRECTIONS)):
    LAMBDA = ptf.CenteredGrid(
        values=lambda loc: A * ptf.math.exp(-ptf.length(loc - LOCATION) ** 2 / (2 * sigma ** 2)),
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    VECTOR_FIELD = ptf.CenteredGrid(
        values=DIRECTION,
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    FORCE_FIELD = FORCE_FIELD + LAMBDA * VECTOR_FIELD


125it [00:03, 37.64it/s]
